<img src="escudo_utfsm.gif" style="float:right;height:100px">
<img src="IsotipoDIisocolor.png" style="float:left;height:100px">
<center>
    <h1> INF285 - Computación Científica</h1>
    <h1> Tarea 2: La descomposición de valores singulares (SVD)</h1> 
    <h2> El fundamento Matemático de PCA (Principal Componene Analysis)</h2>
    <h3> César Quiroz Mansilla 201573578-6</h3>
    <h3> cesar.quirozm@sansano.usm.cl</h3>
</center>

In [1]:
# Librerias
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

## Preguntas

### 1
a) Dado un dataset de $m$ imágenes y utilizando la _Descomposición en Valores Singulares_, explique Matemáticamente cómo podrá obtener los vectores principales y la matriz $P$ de proyección de imágenes sobre el espacio de caras. 
El documento PDF adjunto a esta tarea podría resultar de utilidad.


La Descomposición en Valores Singulares es una factorización de matrices. Que utilizada en caso donde se tienen matrices de nXm con n y m valores muy grandes.

La observación geometrica de esto es:
 
Se considera una circunferencia unitaria que es alargada por valores singulares (σ) en direcciones unitarias ortogonales (u) que será igual a la matriz trabajada multiplicada por vectores principales (v).esto es: 

$$
\text A*V_j = σ_j * u_j
$$

Esto es: 

$$
\text A V = U^ˆ Σ^ˆ
$$

Donde $Σ^ˆ$ es una matriz diagonal con los valores singulares (en su diagonal), $U^ˆ$ posee columnas ortonormales (valores $u_j$) y $V$ es una matriz unitaria con valores de $v_j$ que representa la proyección de imagenes sobre el espacio de caras. Que es lo buscado.  

numpy.linalg.svd, es un comando que permite encontrar estas matrices a partir de una matriz $ A $

Para esta descomposición, lo primero es obtener los valores propios del resultado $A * A^T$. Luego, se obtienen los vectores propios de cada valor propio encontrado, los cuales son normalizados y dispuestos en columnas para formar la matriz V buscada.

Tal como explica el pdf La SVD.



b) Para un dataset de $m$ imágenes, obtenga los vectores principales del espacio de caras, utilizando $k$ componentes principales. Para ello utilice la función `numpy.linalg.svd`. La función solictada es la siguiente:


In [87]:
# reading images from training dataset
image_list = list()
for i in range(1,11):
    img = Image.open("sample/"+str(i)+".pgm")
    
    data = img.getdata()
    data = np.asarray(data)
    image_list.append(data)

#convertir a array
image_list = np.asarray(image_list)

# image matrix
mean = image_list.mean(axis=0)
data = image_list-mean[None,:]
data=data/255 

In [89]:
def svd_pca_components(k, data):
    # Valor singular de descomposición
    u, s, vh = np.linalg.svd(data, full_matrices=False)
    #s -> vector con los valores singulares
    print ("Las dimensiones son: \n", "Matriz u: ", u.shape,"\n Matriz s: ", s.shape, "\n Matriz vh: ", vh.shape)
    l= len(vh[0])
    vh=np.transpose(vh)
    V = np.full((l,k),np.nan)
    
    i=0
    while i< l:
        j=0
        while j<k:
            V[i][j]=vh[i][j]
            j=j+1
        i=i+1
    return V
#K son elementos conponentes importantes.
matriz=svd_pca_components(4,data)
print("matriz reducida", matriz.shape)

Las dimensiones son: 
 Matriz u:  (10, 10) 
 Matriz s:  (10,) 
 Matriz vh:  (10, 10304)
matriz reducida (10304, 4)


### 2

a) Considere el dataset orl\_faces que se compone de $m = 400$ imágenes, el cual se encuentra separado en grupos de $S = 10$ imágenes que corresponden una misma persona. En otras palabras, el dataset contiene 10 imágenes de 40 personas distintas.

Desarrolle un algoritmo que dada la métrica $\text{d}(\cdot,\cdot)$ determine las 2 personas más parecidas y las 2 personas menos parecida de la base de datos entregada.

b) Construya un widget con la función anterior, el cual debe contener varios valores de $k$ y el input `flag`. El resultado gráfico debe mostrar la primera imagen de las personas seleccionadas. ¿Qué puede decir al utilizar distintos valores para $k$? 

In [90]:
# Funcion que calcula la diferecnai entre dos vectores
def diferencia_v_i(v1,v2):
    diferencia=0   
    su=np.linalg.norm(v1+v2)
    re=np.linalg.norm(v1-v2)
    if su<re:
        diferencia+=su
    else:
        diferencia+=re
    return diferencia

In [91]:
# se crean 40 matrices
def creacion_matriz():
    todas= list()
    for carpeta in range(1,41):
        image_list = list()
        for i in range(1,11):
            img = Image.open("orl_faces/s"+str(carpeta)+"/"+str(i)+".pgm")
            data = img.getdata()
            data = np.asarray(data)
            image_list.append(data)
        #convertir a array
        image_list = np.asarray(image_list) 
        # image matrix
        mean = image_list.mean(axis=0)
        data = image_list-mean[None,:]
        data=data/255  # matriz A 
        u, s, vh = np.linalg.svd(data, full_matrices=False)
        l= len(vh[0])
        vh=np.transpose(vh)
        todas.append(vh)
    return todas


infor=creacion_matriz()

In [96]:
# creacion de un arreglo con el formato:
# resultado= [[[k distancias],id1,id2],....] entrega todas las k distancias entre dos imagenes, tiene un largo de 780
def find(k, informacion= infor):
    resultado = list()
    for matriz in range(0,40):
        for matriz2 in range(matriz+1, 40):
            valor=0
            valores= list()
            for vector in range (0,k):
                v1=informacion[matriz][ : ,vector]
                v2=informacion[matriz2][ : ,vector]
                valor+= diferencia_v_i(v1,v2)
                valores.append(valor)
            resultado.append([valores,matriz+1,matriz2+1])
    return resultado
resul=find(k=10, informacion=infor)

In [97]:
def find_people(k=3,flag=0):
    data=resul
    if flag==0:
        me=200000
    else:
        me= -1
    
    resultado=[0,0,0]
    for i in range(0,len(data)):
        if data[i][0][k-1]< me and flag== 0:
            me= data[i][0][k-1]
            resultado=[me,data[i][1],data[i][2]]
        if data[i][0][k-1]> me and flag== 1:
            me= data[i][0][k-1]
            resultado=[me,data[i][1],data[i][2]]
    img1 = Image.open("orl_faces/s"+str(resultado[1])+"/"+str(1)+".pgm")
    img2 = Image.open("orl_faces/s"+str(resultado[2])+"/"+str(1)+".pgm")
    plt.figure(figsize=(5,5))
    f, subplt = plt.subplots(1,2)
    subplt[0].imshow(img1,cmap='gray')
    subplt[1].imshow(img2,cmap='gray')
    plt.show()     
    return resultado
        
#find_people()
interact(find_people,k=(1,10,1),flag=(0,1,1))

interactive(children=(IntSlider(value=3, description='k', max=10, min=1), IntSlider(value=0, description='flag…

<function __main__.find_people(k=3, flag=0)>

Al aumentar k cambian las personas (fotografias de caras), más parecidas, sin embargo existe una  cara (que se encuentra es la carpeta S9), que esta en 9 de los 10 casos posibles, esto indica que sus parametros (pixeles), son muy comunes.

También, se observa que mientras mayor sea k, se obtienen mejores resultados ya que se hace toman en cuenta una mayor cantidad de vectores principales.

a) Considere la matriz $\widehat{M}^{(k)}$, la cual consiste en la matriz de distancia entre todas las caras disponibles en el *dataset* utilizando $k$ componentes principales.
Para construir una matriz de adyacencia y visualizar la cercanía de las caras mediante un grafo, introduciremos un parámetro de cercanía, denotado por $\delta$, el cual nos permitirá separar las caras que son "cercanas" de aquellas que están "lejanas". Utilizando este parámetro $\delta$, podemos definir la matriz de adyacencia $A^{(k)}$ como:

$$
A^{(k)}_{i,j} =
\begin{cases}
    \widehat{M}^{(k)}_{i,j},\quad &\text{si } \text{d}(V_i,V_j) \leq \delta, \\
    0, \quad &\text{en otro caso}.
\end{cases}
$$

Implemente la función `build_adjacency_matrix`, respetando el formato adjunto, para que retorne la matriz $A^{(k)}$ definida anteriormente. Considere que esta función recibe como parámetro: el nombre del dataset (string), el número $k$ de componentes principales a utilizar y el valor de $\delta$. Considere como datasets para esta pregunta: sample y orl_faces.

In [99]:
def build_adjacency_matrix(k=2, delta=12):
    datos=resul
    contador=0
    arreglo= list()
    for i in range(0,len(datos)):
        arreglo.append(datos[i][0][k-1]) 
    A = np.full((40,40),np.nan)
    for i in range(0,40):
        for x in range(0,40):
            if x==i:
                A[i][x]=0
            if x>i:
                if arreglo[contador]<=delta:
                    A[i][x]=arreglo[contador]
                    A[x][i]=arreglo[contador]
                    contador+=1
                else:
                    A[i][x]=0
                    A[x][i]=0
                    contador+=1                    
    return A
#print(build_adjacency_matrix( 10, 19.5))

interact(build_adjacency_matrix,k=(1,10,1),delta=(0,20,0.5))

interactive(children=(IntSlider(value=2, description='k', max=10, min=1), FloatSlider(value=12.0, description=…

<function __main__.build_adjacency_matrix(k=2, delta=12)>

b) Utilizando la función `show_graph`, disponible abajo de este enunciado, visualice el efecto de $k$ y $\delta$ en las caras encontradas. A partir de estos resultados, concluya si la noción de cercanía del método coincide con la percepción humana de "caras parecidas".

In [101]:
def show_graph(k, delta, id1=0, id2=1):
    subjects=40
    # reading images from dataset
    image_list = list()
    for i in range(1,subjects+1):
        img = Image.open("orl_faces/s"+str(i)+"/1.pgm")           
        data = img.getdata()
        data = np.asarray(data)
        image_list.append(data)
    
    #only works if there are subjects
    if subjects > 0:
        #array conversion
        image_list = np.asarray(image_list)

        A = build_adjacency_matrix( k, delta)
        G = nx.from_numpy_matrix(A)

        plt.figure(figsize=(10,5))

        plt.subplot(131)
        colors = ['cyan'] * A.shape[0]
        colors[id1] = 'b'
        colors[id2] = 'r'
        nx.draw(G, with_labels=True, font_weight='bold', node_color=colors)

        plt.subplot(132)
        plt.title("Blue node")
        ni = np.reshape(image_list[id1,:],(112,92))
        plt.imshow(ni,cmap='gray')

        plt.subplot(133)
        plt.title("Red node")
        ni = np.reshape(image_list[id2,:],(112,92))
        plt.imshow(ni,cmap='gray')

        plt.show()

interact(show_graph,k=IntSlider(min=1,max=10,step=1,value=3),delta=(0,50,0.1),id1=(0,39,1),id2=(0,39,1))

interactive(children=(IntSlider(value=3, description='k', max=10, min=1), FloatSlider(value=25.0, description=…

<function __main__.show_graph(k, delta, id1=0, id2=1)>

#### Referencias

+ PDF proporcionado para la tarea